## Declaration of Authorship

We, [Five Guys v1], confirm that the work presented in this assessment is our own. Where information has been derived from other sources, we confirm that this has been indicated in the work. Where a Large Language Model such as ChatGPT has been used we confirm that we have made its contribution to the final submission clear.

Date: 19 December 2023 

Student Numbers: 22203612, 23026398, 23226186, 23193343, 23186084

# 1. Who collected the data?
<p style="text-align: justify;">
Inside Airbnb: Short-Term Rental Property Information
Ministry of Housing, Communities & Local Government (MHCLG): Total Number of Dwellings in Each Borough
Valuation Office Agency: Average Private Rents in Each Borough
</p >

# 2. Why did they collect it?
<p style="text-align: justify;">
Inside Airbnb offers web-scraped data for users interested in exploring the operational dynamics of Airbnb and uncovering regulatory issues associated with the platform.
MHCLG (Ministry of Housing, Communities & Local Government) and VOA (Valuation Office Agency) are government organizations that, on one hand, need to disclose data on public welfare and economy to the public, and on the other hand, provide data for researchers to study.
</p >

# 3. How was the data collected?  
<p style="text-align: justify;">
Airbnb Short-Term Rentals - Regularly obtained from the Airbnb website via web scraping.

Private Rents, Dwellings - Official statistics, gathered through methods including censuses, surveys, and data collection by third-party service providers.</p >

# 4. How does the method of collection impact the completeness and/or accuracy of its representation of the process it seeks to study, and what wider issues does this raise?
<p style="text-align: justify;">
Inside Airbnb data- Airbnb website anti-climbing mechanism, crawler to get the data
 of the wide and mixed, can not ensure that the data of high-quality.
Government statistical data- the actual difficult to implement to every person, every place, the lack of data is inevitable.
</p >

# 5. What ethical considerations does the use of this data raise? 
<p style="text-align: justify;">
Privacy Rights: Airbnb listings may contain personal information of hosts and guests. When using this data, it is necessary to ensure that individual privacy rights are not infringed upon. Even if the data is publicly accessible, the protection of personal information and the ethical use of such data shoul.
Copyright and Intellectual Property: The data scraped may be subject to copyright protection. When using such data, it is necessary to ensure compliance with copyright laws and relevant intellectual property regulations.
Compliance with Laws and Regulations: In certain jurisdictions, there may be specific legal requirements for the collection and use of data. It is necessary to ensure compliance with all relevant laws and regulations, including data protection statutes.
Responsible Reporting and Dissemination: When using data for analysis, research, or reporting, it is essential to maintain responsibility and transparency, ensuring the accuracy and fairness of the information.

\newpage

# 6. With reference to the data (*i.e.* using numbers, figures, maps, and descriptive statistics), what does an analysis of Hosts and Listing types suggest about the nature of Airbnb lets in London? 

In [11]:
import os
from urllib.parse import urlparse
import contextily as ctx
import folium
import geopandas as gpd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from folium.plugins import MarkerCluster
from geopandas.tools import sjoin
from mpl_toolkits.axes_grid1 import make_axes_locatable
from requests import get
from shapely.geometry import Point

In [3]:
def cache_data(source_url, dest_dir):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    filename = source_url.split("/")[-1].split("?")[0] 
    file_path = os.path.join(dest_dir, filename)

    if not os.path.isfile(file_path):
        response = requests.get(source_url)
        if response.status_code == 200:
            with open(file_path, "wb") as file:
                 file.write(response.content)
        else:
            raise Exception(f"Failed to download {source_url}")

    return file_path

## 6.1 Geographic Distribution Map 

<p style="text-align: justify;">
In London, housing distribution varies. Entire home/apts and private rooms are densely concentrated in central areas, especially in business and tourist spots. Hotels also cluster in these central areas. Shared rooms have a more scattered distribution, with lower density. This indicates a higher market demand for private and whole accommodations (Boyle, 2022).
</p>

![Geographic Distribution Map](Geographic_Distribution_Map.png)

In [4]:
# london_Boroughshapefile
London_borough = gpd.read_file(
    cache_data(
        "https://github.com/ZhengyongLiu/FSDS_GroupAssignment_Data/blob/main/Borough/London_Borough_Excluding_MHW.zip?raw=true",
        os.path.join("data", "geo"),
    ),
    driver="ESRI Shapefile",
)
# CSV
url = "https://raw.githubusercontent.com/OnTheMon/FSDS-project/main/Data/cleaned_data_2023.csv"
df = pd.read_csv(url)

In [ ]:
# Converting coordinates to geographic data and selecting the appropriate coordinate system
df = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326"
).to_crs(London_borough.crs)

# Determine the number of rows and columns of the subgraph
n_room_types = len(df["room_type"].unique())
n_cols = 2  
n_rows = n_room_types // n_cols + (n_room_types % n_cols > 0)

# Create a large enough graphical object
fig, axs = plt.subplots(n_rows, n_cols, figsize=(8 * n_cols, 6 * n_rows))

# Iterate over each room type and create a subgraph for each type
for k, room_type in enumerate(df["room_type"].unique()):
    idf = df[df["room_type"] == room_type]
    room_type_clean = room_type.replace("/", " ")

    # Determine the location of the current subgraph
    ax = axs[k // n_cols, k % n_cols]

    # Plotting data on the current subplot
    London_borough.plot(
        ax=ax, alpha=1, edgecolor="#656565", facecolor="#E1E1E1", linewidth=1
    )
    London_borough.plot(
        ax=ax, alpha=1, edgecolor="#656565", facecolor="none", linewidth=1, zorder=10
    )
    ax.scatter(idf.geometry.x, idf.geometry.y, c="#346BAB", s=1, alpha=1)

    # Setting subgraph styles
    for spine in ax.spines.values():
        spine.set_color("white")
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel(room_type_clean)

plt.tight_layout()
plt.show()

## 6.2 Host type and number of listings
<p style="text-align: justify;">
The frequency histogram of the number of properties owned by a single landlord shows that a large number of landlords own only a small number of properties, which may indicate that this group of landlords are renting out their vacant properties on a part-time basis or occasionally; on the other hand, there are a small number of landlords who control a large number of properties. These may be specialist short-term letting companies or investors. A comparison of the average price of Airbnb listings in various areas of London shows that Airbnb prices are generally higher in high-demand areas (e.g. central London), while prices may be more reasonable in remote or non-commercial tourist hotspots.
</p>

![Figure 2 Host type and number of listings](Host_type_and_number_of_listings.png)

In [ ]:
# Host Name Frequency Histogram
host_name_counts = df['host_name'].value_counts()

# Process host_name_counts into the given grouping
bins = [0, 2, 4, 6, 8, 10, float('inf')]
labels = ['1-2', '3-4', '5-6', '7-8', '9-10', '>10']
grouped_counts = pd.cut(host_name_counts, bins=bins, labels=labels, right=False)

# Calculate the frequency of each grouping
grouped_counts = grouped_counts.value_counts().sort_index()

# Create an image and two subimages
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))  

# First sub-chart: bar chart
colors = ['#835AF1', '#8178F0', '#8097EF', '#8AB6E9', '#A1D7DE', '#B8F7D4']
grouped_counts.plot(kind='bar', color=colors, ax=ax1)

# Setting up chart titles and labels
ax1.set_title('Host Frequency Histogram')
ax1.set_ylabel('Frequency')
ax1.set_xlabel('Number of properties per landlord')

# Second sub-map: geographic data map
joined = sjoin(df, London_borough, how='inner', predicate='within')
joined['price'] = joined['price'].replace('[\$,]', '', regex=True).astype(float)

# Mean
meandf= joined.groupby('NAME')['price'].mean().reset_index()
gdf = London_borough.merge(meandf, on='NAME')
cmap = 'Blues'
norm = mcolors.Normalize(vmin=gdf['price'].min(), vmax=gdf['price'].max())

# Drawing GeoDataFrame
divider = make_axes_locatable(ax2)
cax = divider.append_axes("right", size="5%", pad=0.1)
gdf.plot(column='price', ax=ax2, cmap=cmap, legend=True, cax=cax, alpha=0.8, edgecolor='k')

# Add colour bands
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax2, cax=cax)
ax2.set_title('Average House Price of London Boroughs')
ax2.axis('off')
plt.tight_layout()
plt.show()

## 6.3 Summary
<p style="text-align: justify;">
In summary, Airbnb listings in London are concentrated in the city center and hotspots, catering to various market demands. High prices in these areas benefit landlords while individual hosts find it easy to enter the market. Some landlords, like short-term rental companies, dominate a significant share, potentially impacting the long-term rental market.
</p>

\newpage

# 7. Drawing on your previous answers, and supporting your response with evidence (e.g. figures, maps, and statistical analysis/models), how *could* this data set be used to inform the regulation of Short-Term Lets (STL) in London?
<p style="text-align: justify;">
Since Airbnb was launched, its sharing economy concept has profoundly changed the way people travel.Airbnb has won widespread popularity for its price advantage and flexibility compared to traditional hotels. However, with the rapid growth of Airbnb's accommodation business, it has also triggered widespread public concern about its impact. Our research focuses on the impact of Airbnb's different housing types on the London market as well as on space, with a particular focus on the entire room (whole house rental) and private room (single room rental) types.
</p>

### 7.1 Spatial Distribution Change
<p style="text-align: justify;">
Before embarking on specific analyses, we first need to gain an in-depth understanding of the spatial distribution characteristics of Airbnb listings, the object of study. For this purpose, we adopted the kernel density analysis method, which can effectively reveal the spatial density of point distribution and the distribution characteristics within the neighbourhood. Through kernel density analysis, we are not only able to derive the densities of Airbnb listings in different areas, but also observe the trends of these distributions over time.
</p>

![Entire Rental Housing](Entire_Room_Distribution.png)

<p style="text-align: justify;">
Following kernel density calculations and visual analyses, the results reveal a significant increase in the number of Airbnb listings in London between 2016 and 2019. In the city centre, the density of Airbnb's has increased significantly, while on the edge of the city, there is also a trend of continuous expansion into the suburbs. This suggests that Airbnb has experienced a long period of sustained expansion in London.
</p>

from io import BytesIO
from zipfile import ZipFile
from scipy.stats import gaussian_kde
from matplotlib.colors import LinearSegmentedColormap

# data urls
data_urls = {
    '2016': 'https://raw.githubusercontent.com/ZhengyongLiu/FSDS_GroupAssignment_Data/main/Data/cleaned_data_2016.csv',
    '2019': 'https://raw.githubusercontent.com/ZhengyongLiu/FSDS_GroupAssignment_Data/main/Data/cleaned_data_2019.csv'
}

# read shapefile
shapefile_url = 'https://github.com/ZhengyongLiu/FSDS_GroupAssignment_Data/blob/main/Borough/London_Ward.zip?raw=true'
r = requests.get(shapefile_url)
z = ZipFile(BytesIO(r.content))
z.extractall('/tmp/geo')  
london_vector_map = gpd.read_file('/tmp/geo', driver='ESRI Shapefile')
london_vector_map = london_vector_map.to_crs(epsg=4326)

fig, axes = plt.subplots(1, 2, figsize=(16, 12))

# Filter years
for i, year in enumerate(data_urls.keys()):
    # Read data
    df = pd.read_csv(data_urls[year])
    geometry = gpd.points_from_xy(df.longitude, df.latitude)
    geo_df = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')

    # KDE
    xy = np.vstack([df['longitude'], df['latitude']])
    kde = gaussian_kde(xy, bw_method='silverman')
    kde_values = kde(xy)

    # Creating a colour map
    colors = [(0, 'green'), (0.5, 'yellow'), (1, 'red')]
    cmap = LinearSegmentedColormap.from_list('custom_cmap', colors)

    # Normalise and map density values to colours
    norm_kde_values = kde_values / max(kde_values)
    colors = cmap(norm_kde_values)

    # plot map
    ax = axes[i]
    london_vector_map.plot(ax=ax, color='lightgrey', edgecolor='black', linewidth=0.1)
    geo_df.plot(ax=ax, marker='o', color=colors, markersize=0.25)

    # year label
    ax.text(0.5, -0.1, f"Year {year}", size=12, ha='center', transform=ax.transAxes)
    ax.set_axis_off()

plt.tight_layout()
plt.savefig('Entire_Room_Distribution.png', dpi=300)
plt.show()

\newpage

## 7.2 Observation from the Perspective of Entire Room
<p style="text-align: justify;">

This study focuses on analysing the composition and revenue profile of the ENTIRE room type of listings in Airbnb. The so-called entire room refers to a whole house for rent, and the rise of Airbnb has led to many landlords who originally belonged to the traditional long-term rental market, attracted by the high profits of the platform, gradually withdrawing from the housing supply market in London and placing their properties for short-term rentals on Airbnb instead. 

The research methodology is as follows:

1. 2016 and 2019 were chosen as the time points for analysis in this study, given data completeness, due to the fact that London's private housing rental statistics and property ownership data are only updated to 2019.
2. Borough-level analysis to maintain consistency. Extracted Airbnb entire room listings using specific code from insideairbnb. Estimated revenue using an average of 50 annual rental days per listing.
</p>

<p style="text-align: justify;">
The "Person_per_dwelling" data file cited in our study is derived from the official London Dataset [London Dataset](https://data.london.gov.uk/dataset/net- additional-dwellings-borough), which details the total number of dwellings in each London borough. This data reflects to some extent the overall home ownership of the London community and provides us with an important macro perspective.
</p>

![Proportion and income](Proportion_and_income.png)

<p style="text-align: justify;">
The results show that Airbnb, despite representing only 0.5% of properties in most areas, generates high incomes, especially since 2016. This attracts many landlords, leading to a loss of properties in the traditional rental market. Between 2016 and 2023, Entire room listings on Airbnb increased, even rebounding after the pandemic. This indicates a shift from long-term to short-term rentals. This trend affects supply and may raise rental prices, impacting housing market stability. Airbnb's rapid growth has multifaceted effects on the London housing market beyond just numerical changes.
</p>

In [29]:
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import Point
from zipfile import ZipFile
from io import BytesIO
# Set the base path as the current working directory
base_path = os.getcwd()

# Directory for data
data_directory = os.path.join(base_path, 'Data')
if not os.path.exists(data_directory):
    os.makedirs(data_directory)


data_urls = {
    '2016': 'https://raw.githubusercontent.com/ZhengyongLiu/FSDS_GroupAssignment_Data/main/Data/cleaned_data_2016.csv',
    '2019': 'https://raw.githubusercontent.com/ZhengyongLiu/FSDS_GroupAssignment_Data/main/Data/cleaned_data_2019.csv'
}


# Download and extract shapefile for London boroughs
shapefile_url = 'https://github.com/ZhengyongLiu/FSDS_GroupAssignment_Data/blob/main/Borough/London_Borough_Excluding_MHW.zip?raw=true'
r = requests.get(shapefile_url)
z = ZipFile(BytesIO(r.content))
z.extractall('/tmp/geo')  # Extract to a temporary directory
london_vector_map2 = gpd.read_file('/tmp/geo', driver='ESRI Shapefile')
london_vector_map2 = london_vector_map.to_crs(epsg=4326)

# Process data for the specified years
years = [2016, 2019]

for year in years:
    # Read the data for the current year
    df = pd.read_csv(data_urls[str(year)])

    # Filter data with room_type Entire home/apt
    df = df[df['room_type'] == 'Entire home/apt']

    # Processes the price field, removes the '$' symbol and converts it to a floating point number
    df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)

     # Creating a GeoDataFrame from Latitude and Longitude Columns
    geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
    geo_df = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')

    # Perform a spatial connection to associate each point with the corresponding region
    joined = gpd.sjoin(geo_df, london_vector_map2, how="inner", op="within")

    # Calculate point and price averages for each region
    group_data = joined.groupby('GSS_CODE').agg({'price': ['count', 'mean']})
    group_data.columns = ['COUNT', 'price_ave']

    # Calculation of annual income
    group_data['airbnb_annual_income'] = group_data['price_ave'] * group_data['COUNT'] *50

    # Get the name of each area
    borough_names = london_vector_map2.set_index('GSS_CODE')['NAME'].to_dict()

    # merge data
    borough_data = pd.DataFrame({
        'GSS_CODE': group_data.index,
        'NAME': [borough_names[code] for code in group_data.index],
        'COUNT': group_data['COUNT'],
        'price_ave': group_data['price_ave'],
        'airbnb_annual_income': group_data['airbnb_annual_income']
    })

    borough_data.to_csv(os.path.join( f'Data/borough_data_{year}.csv'), index=False)

In [31]:
import pandas as pd

# data download
data_urls = {
    '2016': 'https://raw.githubusercontent.com/ZhengyongLiu/FSDS_GroupAssignment_Data/main/Data/cleaned_data_2016.csv',
    '2019': 'https://raw.githubusercontent.com/ZhengyongLiu/FSDS_GroupAssignment_Data/main/Data/cleaned_data_2019.csv'
}

# Read persons_per_dwelling_df_url
persons_per_dwelling_df_url = "https://raw.githubusercontent.com/ZhengyongLiu/FSDS_GroupAssignment_Data/main/Data/Persons_per_dwelling.csv"
persons_per_dwelling_df = pd.read_csv(persons_per_dwelling_df_url)

years = [2016, 2019]
merged_data = {}  

for year in years:
    # Read each year's borough_data file, correcting the path here
    borough_data_df = pd.read_csv(os.path.join(base_path, f'Data/borough_data_{year}.csv'))
    filtered_persons_per_dwelling = persons_per_dwelling_df[persons_per_dwelling_df['Year'] == year]

    # Merge data based on GSS_CODE and Code
    merged_df = borough_data_df.merge(filtered_persons_per_dwelling, left_on='GSS_CODE', right_on='Code')

    # Store merged data
    merged_data[year] = merged_df
    merged_df.to_csv(os.path.join(base_path, f'Data/borough_data_{year}.csv'), index=False)

In [30]:
file_paths = {
    'borough_data_2016': os.path.join(base_path, 'Data/borough_data_2016.csv'),
    'borough_data_2019': os.path.join(base_path, 'Data/borough_data_2019.csv'),
    'voa_rent_2016': 'https://raw.githubusercontent.com/ZhengyongLiu/FSDS_GroupAssignment_Data/main/Data/voa-rent-2016.csv', 
    'voa_rent_2019': 'https://raw.githubusercontent.com/ZhengyongLiu/FSDS_GroupAssignment_Data/main/Data/voa-rent-2019.csv',
}

data = {}
for key, path in file_paths.items():
    data[key] = pd.read_csv(path)

# Processing 2016 and 2019 data
for year in ['2016', '2019']:
    # Calculate the rent_income_sum column
    data[f'voa_rent_{year}']['rent_income_sum'] = data[f'voa_rent_{year}']['Count of rents'] * data[f'voa_rent_{year}']['Average'] * 12
    # Connecting borough_data and voa_rent data
    data[f'borough_data_{year}'] = pd.merge(data[f'borough_data_{year}'], data[f'voa_rent_{year}'][['Code', 'rent_income_sum']], left_on='GSS_CODE', right_on='Code', how='inner')
    data[f'borough_data_{year}'].to_csv(file_paths[f'borough_data_{year}'], index=False)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches

# def color
light_yellow = (255/255, 255/255, 217/255)
light_green = (215/255, 239/255, 179/255)
teal = (114/255, 200/255, 189/255)
dark_blue = (36/255, 152/255, 193/255)

# Colour mapping function
def apply_color_map(value):
    if value <= 0.1:
        return 'light_yellow'
    elif 0.1 < value <= 0.4:
        return 'light_green'
    elif 0.4 < value <= 1.5:
        return 'teal'
    else:
        return 'dark_blue'
    
# Creating legend labels
light_yellow_patch = mpatches.Patch(color=light_yellow, label='<=0.1')
light_green_patch = mpatches.Patch(color=light_green, label='0.1-0.4')
teal_patch = mpatches.Patch(color=teal, label='0.4-1.5')
dark_blue_patch = mpatches.Patch(color=dark_blue, label='>1.5')

years = [2016, 2019]
fig, axs = plt.subplots(len(years), 2, figsize=(15, 6*len(years)))

for i, year in enumerate(years):
    csv_file = os.path.join(base_path, f"Data/borough_data_{year}.csv")
    borough_data = pd.read_csv(csv_file)
    gdf = gpd.read_file(
    cache_data(
        "https://github.com/ZhengyongLiu/FSDS_GroupAssignment_Data/blob/main/Borough/London_Borough_Excluding_MHW.zip?raw=true",
        os.path.join("data", "geo"),
    ),
    driver="ESRI Shapefile",)

    # A in {year}
    borough_data['airbnb_ratio'] = borough_data['COUNT'] / borough_data['Dwellings'] * 100
    merged_gdf = gdf.merge(borough_data, left_on='GSS_CODE', right_on='GSS_CODE')
    merged_gdf['color'] = merged_gdf['airbnb_ratio'].apply(apply_color_map)
    for color in ['light_yellow', 'light_green', 'teal', 'dark_blue']:
        color_data = merged_gdf[merged_gdf['color'] == color]
        color_data.plot(ax=axs[i, 0], color=eval(color), edgecolor='darkgrey')
    axs[i, 0].set_facecolor('gray')
    axs[i, 0].set_xticks([])
    axs[i, 0].set_yticks([])
    axs[i, 0].set_title(f"Propotion of Entire Rooms in {year}")
    axs[i, 0].legend(handles=[light_yellow_patch, light_green_patch, teal_patch, dark_blue_patch], loc='lower left')

    # B in {year}
    borough_data['airbnb_ratio'] = borough_data['airbnb_annual_income'] / borough_data['rent_income_sum'] * 100
    merged_gdf = gdf.merge(borough_data, left_on='GSS_CODE', right_on='GSS_CODE')
    merged_gdf['color'] = merged_gdf['airbnb_ratio'].apply(apply_color_map)
    for color in ['light_yellow', 'light_green', 'teal', 'dark_blue']:
        color_data = merged_gdf[merged_gdf['color'] == color]
        color_data.plot(ax=axs[i, 1], color=eval(color), edgecolor='darkgrey')
    axs[i, 1].set_facecolor('gray')
    axs[i, 1].set_xticks([])
    axs[i, 1].set_yticks([])
    axs[i, 1].set_title(f"Entire Room Income Ratio in {year}")
    axs[i, 1].legend(handles=[light_yellow_patch, light_green_patch, teal_patch, dark_blue_patch], loc='lower left')

plt.tight_layout()
plt.show()

## 7.3 Observation from the Perspective of Private Room

<p style="text-align: justify;">
This part of the research examines two main types of Airbnb listings: whole house rentals and "private room" listings, with a focus on the latter, where hosts rent out private rooms in their homes. We found that some hosts list multiple private rooms to bypass the 90-day limit, impacting the traditional rental market. In 2023, we conduct spatial cluster analyses on this room type, using criteria like host_listings_count > 1, room_type as private room, and availability_365 > 90 days to select our data.
</p>

![HDBSCAN of London](HDBSCAN.png)

<p style="text-align: justify;">
In the HDBSCAN analysis, setting min_cluster_size to 50 means that regions with fewer than 50 data points will be labeled as noise or individual data points. Ultimately, we obtained 11 clusters, with some clusters distributed not only in the central areas of London mentioned above but also in Brent, Hillingdon. From the results of the HDBSCAN, we can observe a clustering trend of such private rooms in the mentioned areas, indicating a more pronounced impact on the traditional rental housing market. The emergence of this type of housing would be in direct competition with the traditional rental market.

London's approval of [coach house](https://london.ca/sites/default/files/2020-11/2020-11-30-PEC-OZ-9176-Addl-Res-Units-AODA%20%28002%29.pdf) which has same feature of 'private room' we defined, it aimed to increase affordable housing, not to fuel tourist accommodations or corporate ghost hotels. Allowing Airbnb to use these suites contradicts this purpose, risking a reduction in available long-term rentals for low and moderate-income families. Homeowners, tempted by higher short-term rental profits, might withdraw these suites from the long-term market, especially where Airbnb is most lucrative.

Airbnb claims that this shift won't impact long-term rentals, suggesting these suites might not be offered for long-term rent anyway. However, this overlooks why homeowners would create legal secondary suites initially. Moreover, there's nothing stopping owners from preferring short-term rentals for higher profits.
</p>

In [ ]:
# load the data
import hdbscan

url = 'https://raw.githubusercontent.com/ZhengyongLiu/FSDS_GroupAssignment_Data/main/Questions/Question%207%20Sun/cleaned_data_2023.csv'
df = pd.read_csv(url)

#filter out ghost room data (room_type=Private room)
filtered_data = df[df['room_type'] == 'Private room']
# get 13375 ghost room data of 37242 data
#Then tranform coordinate into point and make it GeoDataframe
df_with_coordinates = pd.DataFrame(filtered_data)
geometry = [Point(lon, lat) for lon, lat in zip(df_with_coordinates['longitude'], df_with_coordinates['latitude'])]
gdf_with_coordinates = gpd.GeoDataFrame(df_with_coordinates, geometry=geometry, crs="EPSG:4326")

london_map = gpd.read_file(
    cache_data('https://github.com/ZhengyongLiu/FSDS_GroupAssignment_Data/blob/main/Borough/London_Borough_Excluding_MHW.zip?raw=true', 
               os.path.join('data','geo')), driver='ESRI Shapefile')
london_vector_map = london_map.to_crs('EPSG:4326')
# spatial connection 
merged_gdf = gpd.sjoin(gdf_with_coordinates, london_vector_map, how='left', op='within')


# Next step we do the HDBscan
# create feature dataframe
filter_points = merged_gdf[merged_gdf['host_listings_count']>1]
filtered_points=filter_points[filter_points['availability_365']>90]
features =filtered_points[['longitude', 'latitude']]
HDBscan = hdbscan.HDBSCAN(min_cluster_size=50, metric='haversine')

# cluster the feature
clusters = HDBscan.fit_predict(features)

# add cluster results into GeoDataFrame 
filtered_points['cluster'] = clusters
# Filtering out clustered points.
clustered_points =filtered_points[filtered_points['cluster'] != -1]

# draw the map
plt.figure(figsize=(12,10))
clustered_points.plot(column= 'cluster', cmap='viridis', legend=True, markersize=8, edgecolor='none', alpha=0.7)
plt.title('HDBSCAN Clustering on GeoDataFrame')
london_vector_map.plot(ax=plt.gca(), color='none', edgecolor='grey',alpha=0.4)
plt.show()

# show the cluster amount
num_clusters =clustered_points['cluster'].nunique()
print("Number of clusters:", num_clusters)

## 7.4 Trends in the number of listings

<p style="text-align: justify;">
According to the statistical results of 2016-2023 (2017, 2018 data missing), the short-term rental market in London has been maintaining the trend of increasing the number of properties, Entire room and private room overall trend is close to the pandemic brought about by the brief depression, with the flow of population, tourism recovery soon make the number of soon have a new increase. This shows that the presence of Airbnb is constantly leading to a shift from traditional to short-term rentals, which undoubtedly poses a challenge to the stability of supply in the rental market.
</p>

![Count of Entire home_apt by Year](Count_of_Entire_home_or_apt_by_Year.png)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

years = [2016, 2019, 2020, 2021, 2022, 2023]
data_frames = {}  # Create a dictionary to store data frames for different years

# Store the number of 'Entire home/apt' and 'Private room' for each year
entire_home_counts = []
private_room_counts = []

for year in years:
    
    data_frames[f'pd_{year}'] = pd.read_csv(f'https://raw.githubusercontent.com/ZhengyongLiu/FSDS_GroupAssignment_Data/main/Data/cleaned_data_{year}.csv')
    # Count the number of 'Entire home/apt' for each year 
    count_entire_home = data_frames[f'pd_{year}'][data_frames[f'pd_{year}']['room_type'] == 'Entire home/apt']['room_type'].count()
    # Count the number of 'Private room' for each year
    count_private_room = data_frames[f'pd_{year}'][data_frames[f'pd_{year}']['room_type'] == 'Private room']['room_type'].count()
    entire_home_counts.append(count_entire_home)
    private_room_counts.append(count_private_room)

# Define colors
teal = (114/255, 200/255, 189/255)
light_green = (215/255, 239/255, 179/255)

# Plot the 'Entire home/apt' line chart
plt.plot(years, entire_home_counts, marker='o', linestyle='-', color=teal, linewidth=5, label='Entire home/apt')

# Plot the 'Private room' line chart
plt.plot(years, private_room_counts, marker='o', linestyle='-', color=light_green, linewidth=5, label='Private room')

plt.xlabel('Year')
plt.ylabel('Count of Rooms')
plt.title('Count of Rooms by Year')
plt.legend()
plt.grid(axis='y')
plt.show()

## 7.5 Conclusion & Recommendations
<p style="text-align: justify;">
The proposed policy recommendations focus on creating a balanced and sustainable rental market in London, amidst the growing influence of Airbnb. Key measures include establishing regulations to limit the conversion of long-term rental properties into short-term Airbnb listings, particularly in highly impacted areas. To protect affordable housing, policies should enforce the dedication of units like coach houses to long-term rentals, countering their use for short-term tourist accommodations. Additionally, introducing profit-sharing mechanisms or higher taxation on Airbnb profits could help equalize the attractiveness of both rental types. Mandatory data sharing from platforms like Airbnb would aid in effective monitoring and policy formulation. Finally, offering incentives or subsidies to property owners who commit to long-term rentals, especially in areas with high demand, could further support the stability and affordability of the long-term rental market. These strategies aim to harmonize the growth of short-term rental platforms with the broader housing needs of the community.
</p>

\newpage

# References
Inside Airbnb (2016) 'Inside Airbnb: Home', Available at: [Inside Airbnb](http://insideairbnb.com/) (Accessed: 04 December 2023).

Slee, T. (2017). *Airbnb Data Collection: Methodology and Accuracy*. Retrieved from [http://tomslee.net/airbnb-data-collection](http://tomslee.net/airbnb-data-collection)

Stoyanovich, J., Howe, B. & Jagadish, H.V., 2020. Responsible data management. *Proceedings of the VLDB Endowment*, 13(12).

Boyle, M. (2022) 'London Airbnb statistics: How has Airbnb grown in the UK’s capital?, Finder.com. Available at: https://www.finder.com/uk/airbnb-statistics (Accessed: 09 December 2023).

Woods, D. (2020) 'Airbnb is being overrun by ghost hotels', Paper Giant. Available at: https://www.papergiant.net/news/airbnb-is-being-overrun-by-ghost-hotels (Accessed: 11 December 2023).

Mayor of London. (2023). Have your say on short-term lettings. Available at: https://www.london.gov.uk/talk-london/topics/housing/short-term-lettings-london/surveys/926 [Accessed: 18 Dec 2023].

Shabrina, Z., Arcaute, E., & Batty, M. (2022). Airbnb and its potential impact on the London housing market. Urban Studies, 59(1), 197-221. 
https://doi.org/10.1177/0042098020970865

Masson, T. (2023). Decoding Airbnb’s strategic priorities for 2023. Rental Scale-Up. Available at: www.rentalscaleup.com [Accessed 19 Dec. 2023].